In [4]:
import pymongo
import os
from dotenv import load_dotenv
import pandas as pd
import requests
from bs4 import BeautifulSoup


load_dotenv()

DATABASE_URL = os.getenv('DATABASE_URL')
CAR_URL = "https://www.kudosprime.com/fh5/car_sheet.php?id="

In [2]:
db_client = pymongo.MongoClient(DATABASE_URL)
db_col = db_client["fhta"]["Car"]

In [3]:
cars = pd.read_excel("fh5_cars_kudosprime.xlsx")
cars.rename(columns={
    "K' iD":"kId",
    "Group":"group",
    "Maker":"make",
    "Model":"model"
}, inplace=True)
cars

,kId,group,make,model
0,1,RETRO HOT HATCH,Acura,Integra Type R
1,2,RETRO HOT HATCH,Acura,RSX Type S
2,3,MODERN SUPERCARS,Acura,NSX
3,4,CLASSIC RALLY,Alpine,A110 1600s
4,5,MODERN SPORTS CARS,Alpine,A110
...,...,...,...,...
672,675,MODERN SPORTS CARS,Nissan,Z
673,676,RETRO SPORTS CARS,Toyota,MR2 GT
674,677,RETRO RALLY,Mitsubishi,Galant VR-4
675,678,CLASSIC SPORTS CARS,Toyota,Sports 800


In [30]:
cars.isna().sum()

group    2
make     0
model    0
dtype: int64

In [37]:
cars.dropna(inplace=True)
cars

,group,make,model
0,RETRO HOT HATCH,Acura,Integra Type R
1,RETRO HOT HATCH,Acura,RSX Type S
2,MODERN SUPERCARS,Acura,NSX
3,CLASSIC RALLY,Alpine,A110 1600s
4,MODERN SPORTS CARS,Alpine,A110
...,...,...,...
671,SUPER HOT HATCH,Renault,Megane RS 250
672,MODERN SPORTS CARS,Nissan,Z
673,RETRO SPORTS CARS,Toyota,MR2 GT
674,RETRO RALLY,Mitsubishi,Galant VR-4


In [68]:
kId = 1
soup = BeautifulSoup(requests.get(CAR_URL+str(kId)).content, "html.parser")

In [126]:

# year                      Int?
desc = soup.find("meta", attrs={'name':'description'})
desc = desc.attrs["content"].split(",")
year = -1
for row in desc:
    if "Year" in row:
        year = int(row.split(":")[1].strip())

# imageLink                 String?
imageLink = "https://www.kudosprime.com/fh5/images/cars/big/fh5_car_"+  str(kId) + ".jpg?v=1"

# stockPerformancePoints    Int?
stockPerformancePoints = int(soup.find("span", class_="pimain").contents[1].text)

# maxPerformancePoints      Int?
maxPerformancePoints = int(soup.find("span", class_="pimax").contents[1].text)

# stockDrivetrain           String?
stockDrivetrain = soup.find("span", class_="transmission").text

# stockIngamePrice          Int?
stockIngamePrice = int(soup.find("div", class_="price").contents[0].text.replace(",", ""))

# carSource                 String?
carSource = soup.find("div", class_="car_source").contents[0].text

# stockSpeed                Float?
stockSpeed = float(soup.find("div", class_="speed").contents[0].text)

# stockHandling             Float?
stockHandling = float(soup.find("div", class_="handling").contents[0].text)

# stockAcceleration         Float?
stockAcceleration = float(soup.find("div", class_="acceleration").contents[0].text)

# stockLaunch               Float?
stockLaunch = float(soup.find("div", class_="launch").contents[0].text)

# stockBraking              Float?
stockBraking = float(soup.find("div", class_="braking").contents[0].text)

# stockOffroad              Float?
stockOffroad = float(soup.find("div", class_="offroad").contents[0].text)

# stockTopSpeed             Float?
stockTopSpeedKmh = float(soup.find("div", class_="car_perfs").contents[1].text.split(":")[1].split(" ")[0])

# stockLateralGs            Float?
stockLateralGs = float(soup.find("div", class_="car_perfs").contents[3].text.split(":")[1])

# stock0To97                Float?
stock0To97 = float(soup.find("div", class_="car_perfs").contents[5].text.split(":")[1].replace("s", ""))

# stock0To161               Float?
stock0To161 = float(soup.find("div", class_="car_perfs").contents[7].text.split(":")[1].replace("s", ""))

# stockHorsepower           Int?
stockHorsepower = int(soup.find("span", class_="power").contents[0].text)

# stockWeight               Int?
stockWeightLbs = int(soup.find("span", class_="weight").contents[0].text)


2639

In [19]:
for car in cars.rows():
    car
    

AttributeError: 'DataFrame' object has no attribute 'rows'

In [42]:
cars_dict = cars.drop(columns=["kId"]).to_dict("records")
cars_dict

[{'group': 'RETRO HOT HATCH', 'make': 'Acura', 'model': 'Integra Type R'},
 {'group': 'RETRO HOT HATCH', 'make': 'Acura', 'model': 'RSX Type S'},
 {'group': 'MODERN SUPERCARS', 'make': 'Acura', 'model': 'NSX'},
 {'group': 'CLASSIC RALLY', 'make': 'Alpine', 'model': 'A110 1600s'},
 {'group': 'MODERN SPORTS CARS', 'make': 'Alpine', 'model': 'A110'},
 {'group': 'UNLIMITED BUGGIES',
  'make': 'Alumi Craft',
  'model': 'Class 10 Race Car'},
 {'group': 'CULT CARS', 'make': 'AMC', 'model': 'Gremlin X'},
 {'group': 'UNLIMITED OFFROAD',
  'make': 'AMG TRANSPORT DYNAMICS',
  'model': 'M12S Warthog CST'},
 {'group': 'EXTREME TRACK TOYS',
  'make': 'Apollo',
  'model': 'Intensa Emozione'},
 {'group': 'EXTREME TRACK TOYS',
  'make': 'Apollo',
  'model': 'Intensa Emozione "Welcome Pack"'},
 {'group': 'EXTREME TRACK TOYS', 'make': 'Ariel', 'model': 'Atom 500 V8'},
 {'group': 'UNLIMITED BUGGIES', 'make': 'Ariel', 'model': 'Nomad'},
 {'group': 'RARE CLASSICS', 'make': 'Aston Martin', 'model': 'DB5'},
 

In [39]:
db_col.insert_many(cars_dict)